# Best Working model testing for Prediction of Hospital Readmission and it's reason if readmission prediction occurs true


## Note that, since the jupyter notebook doesn't save the inputs passed to the terminal, only the outputs are visible when we saved and uploaded this file to the github. 
## Please refer to the "Readmissions_best_working_models_testing_RESULTS.pdf" file uploaded on the github to see various inputs we tried to test our models.



### Used models-->
### Objective 1 - Readmission Prediction - GRU model
### Objective 1 - Readmission Reason pred - FCNN with Attention mechanism model tuned using keras tuner

In [ ]:
import numpy as np
from scipy.sparse import hstack, csr_matrix
from tensorflow.keras.models import load_model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
import joblib
# Load models
gru_model_path = 'gru_model_latest.h5'  # Objective-1 GRU model
gru_model = load_model(gru_model_path)
fcnn_model_path = "fcnn_attention_tuned_model.pkl"  # Objective-2 FCNN + Attention model
fcnn_attention_model = joblib.load(fcnn_model_path)
# Define thresholds
OBJECTIVE_1_THRESHOLD = 0.3689  # Updated Precision-Recall Threshold for Objective-1
OBJECTIVE_2_THRESHOLDS = [0.57, 0.50, 0.56, 0.40, 0.48, 0.52, 0.59, 0.56, 0.50, 0.49, 0.51]  # Class-specific thresholds for Objective-2
# Initialize preprocessors (same logic and configuration as training)
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
scaler = StandardScaler()
mlb_icd9 = MultiLabelBinarizer()
mlb_proc_icd9 = MultiLabelBinarizer()
mlb_drug_code = MultiLabelBinarizer()
# Simulated fit of preprocessors (Ensure these match training data)
tfidf_vectorizer.fit(["sample text"] * 100)  # Simulating training fit
scaler.fit([[0]] * 100)  # Simulating training fit for numerical features
mlb_icd9.fit([["ICD1", "ICD2"]])  # Replace with actual training classes
mlb_proc_icd9.fit([["PROC1", "PROC2"]])
mlb_drug_code.fit([["DRG1", "DRG2"]])
# Initialize MultiLabelBinarizer for decoding output labels (Objective-2)
correct_classes = [
   'Certain Conditions originating in the Perinatal Period',
   'Diseases of the Blood and Blood-forming Organs',
   'Diseases of the Circulatory System',
   'Diseases of the Digestive System',
   'Diseases of the Genitourinary System',
   'Diseases of the Nervous System and Mental Disorders',
   'Diseases of the Respiratory System',
   'Endocrine, Nutritional and Metabolic Diseases',
   'Infectious and Parasitic Diseases',
   'Injury and Poisoning',
   'Symptoms, Signs and Ill-defined Conditions'
]
mlb = MultiLabelBinarizer(classes=correct_classes)
mlb.fit([])  # Ensure compatibility for inverse transform
# Combined interface function
def predict_readmission_and_reasons():
   print("Welcome to the Readmission and Reasons Prediction System!")
   # Collect user inputs
   cleaned_text = input("Enter the cleaned text data: ")
   past_admissions = float(input("Enter the number of past 6-month hospital admissions: "))
   icd9_codes = input("Enter ICD9 codes (comma-separated): ").split(',')
   proc_icd9_codes = input("Enter Procedure ICD9 codes (comma-separated): ").split(',')
   drg_codes = input("Enter DRG codes (comma-separated): ").split(',')
   # Preprocess inputs for both Objective-1 and Objective-2
   try:
       X_text = tfidf_vectorizer.transform([cleaned_text])
       X_numerical = scaler.transform([[past_admissions]])
       icd9_encoded = mlb_icd9.transform([list(set(icd9_codes) & set(mlb_icd9.classes_))])
       proc_icd9_encoded = mlb_proc_icd9.transform([list(set(proc_icd9_codes) & set(mlb_proc_icd9.classes_))])
       drug_code_encoded = mlb_drug_code.transform([list(set(drg_codes) & set(mlb_drug_code.classes_))])
   except ValueError as e:
       print(f"Error in preprocessing input: {e}")
       return
   # Combine all features into a single sparse matrix
   X_combined = hstack([X_text, X_numerical, icd9_encoded, proc_icd9_encoded, drug_code_encoded])
   # Ensure the feature count matches the model's expectation
   EXPECTED_FEATURE_COUNT_OBJ1 = gru_model.input_shape[-1]  # Objective-1 features
   EXPECTED_FEATURE_COUNT_OBJ2 = fcnn_attention_model.input_shape[1]  # Objective-2 features
   # Padding or trimming for Objective-1
   if X_combined.shape[1] < EXPECTED_FEATURE_COUNT_OBJ1:
       padding = csr_matrix((X_combined.shape[0], EXPECTED_FEATURE_COUNT_OBJ1 - X_combined.shape[1]))
       X_combined_obj1 = hstack([X_combined, padding])
   else:
       X_combined_obj1 = X_combined[:, :EXPECTED_FEATURE_COUNT_OBJ1]
   # Padding or trimming for Objective-2
   if X_combined.shape[1] < EXPECTED_FEATURE_COUNT_OBJ2:
       padding = csr_matrix((X_combined.shape[0], EXPECTED_FEATURE_COUNT_OBJ2 - X_combined.shape[1]))
       X_combined_obj2 = hstack([X_combined, padding])
   else:
       X_combined_obj2 = X_combined[:, :EXPECTED_FEATURE_COUNT_OBJ2]
   # Reshape for GRU (Objective-1)
   X_combined_obj1 = np.array(X_combined_obj1.todense()).reshape(1, 1, -1)
   # Predict Objective-1
   prediction_prob_obj1 = gru_model.predict(X_combined_obj1)[0][0]
   prediction_class_obj1 = int(prediction_prob_obj1 > OBJECTIVE_1_THRESHOLD)
   print("\nObjective-1 Prediction Results:")
   #print(f"Probability of Readmission: {prediction_prob_obj1:.4f}")
   print(f"Will the patient be readmitted? {'Yes' if prediction_class_obj1 == 1 else 'No'}")
   if prediction_class_obj1 == 1:
       print("\nPatient is likely to be readmitted. Predicting reasons (Objective-2)...")
       # Predict Objective-2
       X_combined_obj2 = np.array(X_combined_obj2.todense())
       prediction_prob_obj2 = fcnn_attention_model.predict(X_combined_obj2)[0]
       # Apply class-specific thresholds
       prediction_class_obj2 = (prediction_prob_obj2 > OBJECTIVE_2_THRESHOLDS).astype(int)
       # Decode predictions back to labels
       # Decode predictions back to labels
       predicted_labels = mlb.inverse_transform(np.array([prediction_class_obj2]))

       print("\nObjective-2 Prediction Results:")
       print(f"Predicted Reasons for Readmission: {', '.join(predicted_labels[0]) if predicted_labels else 'None'}")
       #print(f"Prediction Probabilities: {dict(zip(mlb.classes_, prediction_prob_obj2))}")
   else:
       print("\nPatient is not likely to be readmitted. No reasons to predict.")


### Run the function to test on various inputs 


In [134]:
predict_readmission_and_reasons()

Welcome to the Readmission and Reasons Prediction System!


Enter the cleaned text data:  Chief Complaint: The patient presented with mild abdominal pain and occasional nausea over the past week. History of Present Illness: Emily Carter, a 32-year-old female with no significant past medical history, reported mild intermittent abdominal discomfort accompanied by nausea. Symptoms began after consuming a large, spicy meal. No history of fever, diarrhea, or significant weight loss. On evaluation, she was hemodynamically stable, and lab results were within normal limits except for mildly elevated liver enzymes. Abdominal ultrasound revealed mild cholecystitis without gallstones. Physical Examination: General Appearance: Alert, well-nourished, and in no acute distress. Vital Signs: Temperature: 98.7°F Heart Rate: 75 bpm Blood Pressure: 120/78 mmHg Respiratory Rate: 16 breaths per minute Abdominal Exam: Mild tenderness in the right upper quadrant, no rebound or guarding. Diagnostics and Labs: Complete Blood Count (CBC): Normal. Liver Function Tests: A

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step

Objective-1 Prediction Results:
Will the patient be readmitted? No

Patient is not likely to be readmitted. No reasons to predict.


In [104]:
predict_readmission_and_reasons()

Welcome to the Readmission and Reasons Prediction System!


Enter the cleaned text data:  Reason for Admission: Acute exacerbation of chronic obstructive pulmonary disease (COPD) with respiratory distress.  Subjective: The patient, a 65-year-old male with a history of COPD, presented with worsening shortness of breath, productive cough with yellow sputum, and mild fever for the past three days. The patient reported increased fatigue and difficulty performing daily activities.  Objective:      Vital Signs: Slight tachycardia (HR: 104 bpm), SpO2: 88% on room air, improved to 94% on 2L nasal cannula.     Physical Examination:         Lungs: Bilateral wheezing, diminished breath sounds at the bases.         No cyanosis or accessory muscle use observed at rest.     Laboratory Tests:         Elevated WBC count (13,200/mm³).         Arterial blood gas showed pH 7.36, pCO2 56 mmHg, pO2 62 mmHg on 2L oxygen.     Chest X-ray: Hyperinflation of lungs with no new infiltrates.  Assessment: The patient was diagnosed with an acute exacerbation of COPD, likely

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step

Objective-1 Prediction Results:
Probability of Readmission: 0.3535
Will the patient be readmitted? Yes

Patient is likely to be readmitted. Predicting reasons (Objective-2)...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step

Objective-2 Prediction Results:
Predicted Reasons for Readmission: Diseases of the Respiratory System


In [114]:
predict_readmission_and_reasons()

Welcome to the Readmission and Reasons Prediction System!


Enter the cleaned text data:  Chief Complaint: The patient presented with chest pain, shortness of breath, and fatigue, which began two days prior to admission. History of Present Illness: The patient, a 58-year-old male with a history of hypertension and hyperlipidemia, presented with acute chest pain radiating to the left arm, accompanied by shortness of breath. Symptoms occurred at rest and were unrelieved by over-the-counter medications. The patient had experienced similar episodes over the past month but did not seek medical attention earlier. Physical Examination: General Appearance: Pale, anxious, moderate distress. Vital Signs: Blood pressure elevated at 180/98 mmHg, pulse irregular at 105 bpm. Cardiac Examination: Irregular heart rhythm, audible S3 gallop, no significant murmurs. Respiratory System: Bilateral crackles in lung bases, indicating possible pulmonary congestion. Extremities: Mild bilateral pedal edema. Diagnostics and Labs: ECG: ST-segment depression in anterior le

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step

Objective-1 Prediction Results:
Probability of Readmission: 0.3535
Will the patient be readmitted? Yes

Patient is likely to be readmitted. Predicting reasons (Objective-2)...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step

Objective-2 Prediction Results:
Predicted Reasons for Readmission: Diseases of the Circulatory System


In [130]:
predict_readmission_and_reasons()

Welcome to the Readmission and Reasons Prediction System!


Enter the cleaned text data:  Chief Complaint: The patient was admitted with severe fatigue, persistent nausea, confusion, and significant electrolyte imbalances. History of Present Illness: Mark Johnson, a 58-year-old male with a history of poorly controlled type 2 diabetes mellitus and chronic kidney disease, presented to the emergency department with worsening lethargy and confusion. His family reported reduced oral intake and recurrent vomiting over the last 48 hours. Initial labs revealed severe hyperosmolar hyperglycemic state (HHS), hypokalemia, and metabolic acidosis. Physical Examination: General Appearance: Drowsy, mildly dehydrated, diaphoretic. Vital Signs: Temperature: 99.5°F Heart Rate: 110 bpm Blood Pressure: 88/52 mmHg Respiratory Rate: 22 breaths per minute Endocrine Exam: Dry mucous membranes; no thyroid enlargement. Neurological Exam: Reduced alertness; no focal deficits. Cardiac Exam: Regular rhythm with tachycardia. Diagnostics and Labs: Blood Glucose: 825 mg/dL. S

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step

Objective-1 Prediction Results:
Will the patient be readmitted? Yes

Patient is likely to be readmitted. Predicting reasons (Objective-2)...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step

Objective-2 Prediction Results:
Predicted Reasons for Readmission: Endocrine, Nutritional and Metabolic Diseases


In [132]:
predict_readmission_and_reasons()

Welcome to the Readmission and Reasons Prediction System!


Enter the cleaned text data:  Chief Complaint: The patient presented with mild abdominal pain and occasional nausea over the past week. History of Present Illness: Emily Carter, a 32-year-old female with no significant past medical history, reported mild intermittent abdominal discomfort accompanied by nausea. Symptoms began after consuming a large, spicy meal. No history of fever, diarrhea, or significant weight loss. On evaluation, she was hemodynamically stable, and lab results were within normal limits except for mildly elevated liver enzymes. Abdominal ultrasound revealed mild cholecystitis without gallstones. Physical Examination: General Appearance: Alert, well-nourished, and in no acute distress. Vital Signs: Temperature: 98.7°F Heart Rate: 75 bpm Blood Pressure: 120/78 mmHg Respiratory Rate: 16 breaths per minute Abdominal Exam: Mild tenderness in the right upper quadrant, no rebound or guarding. Diagnostics and Labs: Complete Blood Count (CBC): Normal. Liver Function Tests: A

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step

Objective-1 Prediction Results:
Will the patient be readmitted? No

Patient is not likely to be readmitted. No reasons to predict.


In [133]:
predict_readmission_and_reasons()

Welcome to the Readmission and Reasons Prediction System!


Enter the cleaned text data:  Chief Complaint: The patient reported mild respiratory discomfort, intermittent cough, and nasal congestion, persisting for three days. History of Present Illness: The patient, a 32-year-old female with a history of seasonal allergies and a recent upper respiratory infection, presented with mild respiratory symptoms that developed after exposure to cold weather. The symptoms are consistent with mild upper respiratory irritation and do not indicate a more severe underlying issue. The patient denies fever, chest pain, or significant shortness of breath. Physical Examination: General Appearance: Alert and oriented, no acute distress. Vital Signs: Stable. Respiratory System: Clear breath sounds bilaterally, mild wheezing noted in upper airways. ENT: Mild nasal inflammation and congestion, throat mildly erythematous. Abdomen: Soft, non-tender, no organomegaly. Diagnostics and Labs: CBC: Normal limits. Chest X-Ray: Clear, no signs of pneumonia or fluid collectio

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step

Objective-1 Prediction Results:
Will the patient be readmitted? No

Patient is not likely to be readmitted. No reasons to predict.
